In [4]:
import pandas as pd
import sqlite3
import os

# --- Ruta absoluta a la carpeta de datos ---
carpeta_datos = r"C:\Users\pablo\App_Pisos\data"

# --- Rutas de archivos ---
archivo_csv_alquiler = os.path.join(carpeta_datos, "alquiler.csv")
archivo_csv_ventas   = os.path.join(carpeta_datos, "ventas.csv")
db_alquiler          = os.path.join(carpeta_datos, "database_alquiler.sql")
db_ventas            = os.path.join(carpeta_datos, "database_ventas.sql")

# --- Columnas esperadas ---
columnas_esperadas = [
    "titulo", "precio", "ubicacion", "habitaciones", "baños", "metros",
    "superficie_construida", "superficie_util", "tipo_vivienda",
    "descripcion_ampliada", "link", "precio_m2"
]

# --- Columnas booleanas para tipo de vivienda ---
lista_tipos_vivienda = ["piso", "atico", "estudio", "apartamento", "duplex", "chalet", "finca", "loft"]

# --- Generar tipo_vivienda desde columnas booleanas ---
def generar_tipo_vivienda(df):
    df = df.copy()

    boolean_matrix = df[lista_tipos_vivienda].fillna(False).astype(bool)
    suma_booleans = boolean_matrix.sum(axis=1)

    conflictivas = df[suma_booleans > 1]
    if not conflictivas.empty:
        print(f"⚠️ {len(conflictivas)} filas tienen múltiples tipos de vivienda marcados. Se eliminan.")
        df = df[suma_booleans <= 1]

    df["tipo_vivienda"] = boolean_matrix.to_numpy().argmax(axis=1)
    df["tipo_vivienda"] = df["tipo_vivienda"].apply(lambda val: lista_tipos_vivienda[val])
    return df

# --- Preparar DataFrame final ---
def preparar_dataframe(ruta_csv):
    if not os.path.exists(ruta_csv):
        raise FileNotFoundError(f"No se encuentra: {ruta_csv}")

    df = pd.read_csv(ruta_csv)

    if all(col in df.columns for col in lista_tipos_vivienda):
        df = generar_tipo_vivienda(df)

    for col in columnas_esperadas:
        if col not in df.columns:
            df[col] = pd.NA

    return df[columnas_esperadas]

# --- Guardar DataFrame en SQLite ---
def guardar_en_sqlite(df, ruta_sqlite):
    conn = sqlite3.connect(ruta_sqlite)
    df.to_sql("anuncios", conn, if_exists="replace", index=False)
    conn.close()
    print(f"✅ Base de datos guardada en: {ruta_sqlite}")
    print(df.head())  # Vista previa

# --- Ejecutar para ambas bases ---
df_alquiler = preparar_dataframe(archivo_csv_alquiler)
guardar_en_sqlite(df_alquiler, db_alquiler)

df_ventas = preparar_dataframe(archivo_csv_ventas)
guardar_en_sqlite(df_ventas, db_ventas)


✅ Base de datos guardada en: C:\Users\pablo\App_Pisos\data\database_alquiler.sql
                                 titulo  precio         ubicacion  \
0          Piso en calle de Gravina, 15  1990.0   Justicia-Chueca   
1         Piso en Avenida de los Prunos  1600.0           Palomas   
2  Dúplex en calle del Cardenal Mendoza  1750.0  Puerta del Ángel   
3                 Piso en calle Lagasca  3500.0        Castellana   
4             Piso en Paseo de Pontones  1700.0          Imperial   

   habitaciones  baños  metros  superficie_construida superficie_util  \
0           2.0    2.0    55.0                   55.0            <NA>   
1           2.0    2.0    93.0                   93.0            <NA>   
2           2.0    1.0   130.0                  130.0            <NA>   
3           2.0    2.0    92.0                   92.0            <NA>   
4           2.0    2.0    88.0                   88.0            <NA>   

  tipo_vivienda descripcion_ampliada  \
0          piso          